In [1]:
import numpy as np
import itertools
import picos as pic
import Fun as f
import mosek
import sys

import collections

In [5]:
def SDP(n,K,d,D):
    
    ##### SDP #####

    pic.available_solvers()
    
    Ap = pic.RealVariable("Ap", n+1)  # A column vector with n+1 elements.
    
    sdp = pic.Problem()
    
    #sdp.options["abs_*_tol"] = 1e-06
    
    sdp.set_objective('find') 
                   
    ##### Adding matrix constraints #####
    
    ##### NPA #####
     
    ### Constructing Beta matrices ###

    A=[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 18216.0, 
       0.0, 156492.0, 0.0, 1147608.0, 0.0, 3736557.0, 0.0, 6248088.0, 0.0, 4399164.0, 0.0, 1038312.0, 0.0, 32778.0]
    
    Ap2=[]
    
    for m in range(n+1):
    
        Ap2.append(sum(D**(-m)*f.comb(n-j,n-m)*A[j] for j in range(m+1)))

    Bmatrix=f.BetaMatrices(Ap,n) #### B_j(x)>>0 ####
    
    sdp.add_list_of_constraints(Ap[k]==Ap2[k] for k in range(int(n/2)+1))

    sdp.add_list_of_constraints(pic.block(Bmatrix[k])>>0 for k in range(int(n/2)+1))
        
    ##### Adding K-L constrains #####
    
    ### K*B'_j-A'_j=0  for d > j >= 0 ###
    
    sdp.add_list_of_constraints((K*Ap[n-i]-Ap[i])== 0 for i in range(d)) 
    
    ### A_j>=0 ###
    
    for m in range(n+1):
        A=sum((-1)**(m-j)*D**(j)*f.comb(n-j,n-m)*Ap[j] for j in range(m+1))
        sdp.add_constraint(A >= 0)

    ### K*B_j-A_j>=0 ###
    
    for m in range(n+1):   
                                    
        KBA=sum((-1)**(m-j)*D**(j)*f.comb(n-j,n-m)*(K*Ap[n-j]-Ap[j]) for j in range(m+1))
        sdp.add_constraint(KBA >= 0)
    
    ##### Adding Shadow constrains #####
    
    ### sum(K_j*A'_j)>=0 ###
    
    for j in range(n+1):
        
        S=sum(f.K_fun(n-j,i,n)*Ap[i] for i in range(n+1))
        sdp.add_constraint(S >= 0)
    
     
    ### Type II qubit codes  ###
    
    suma=0
        
    for m in range(0,n+1):
        
        if (m % 2) == 0:
            
            A=sum((-1)**(m-j)*D**(j)*f.comb(n-j,n-m)*Ap[j] for j in range(m+1))
        
            suma += A
        
    sdp.add_constraint(suma == 2**(n-int(round(np.log2(K)))))

    
    ### Convention for K=1 codes ###
    
    if K==1:
        
        sdp.add_list_of_constraints(Ap[i]== f.comb(n,i)/D**i for i in range(1,d))
    
    sol = sdp.solve(primals=True,solver='mosek')
    
    a=[]
    
    for m in range(n+1):
        A=sum((-1)**(m-j)*D**(j)*f.comb(n-j,n-m)*Ap[j] for j in range(m+1))
        a.append(np.round(A,2))
        #a.append(np.round(A,8))
    
    print(a)
    
    if sol.problemStatus == "feasible":
        task=sdp.strategy.solver.int
        task.set_Stream(mosek.streamtype.msg, sys.stdout.write)
        print(task.solutionsummary(mosek.streamtype.msg))     
        
        #print(task.getsolution(mosek.soltype.itr)[3])
        #print(task.getsolution(mosek.soltype.itr)[4])
        #print(task.getprimalsolutionnorms(mosek.soltype.itr))
        return True    
    
    elif sol.problemStatus == "infeasible":
        #task=sdp.strategy.solver.int
        #task.set_Stream(mosek.streamtype.msg, sys.stdout.write)
        #print(task.solutionsummary(mosek.streamtype.msg))
        
        return False
    else:
        #task=sdp.strategy.solver.int
        #task.set_Stream(mosek.streamtype.msg, sys.stdout.write)
        #print(task.solutionsummary(mosek.streamtype.msg))
        
        return sol.problemStatus 

D=2
Codes=[[24,2**0,10]]

for codes in Codes:
    print("%s=%s" % (codes,SDP(codes[0],codes[1],codes[2],D)))

[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 18216.0, 0.0, 156492.0, 0.0, 1147608.0, 0.0, 3736557.0, 0.0, 6248088.0, 0.0, 4399164.0, 0.0, 1038312.0, 0.0, 32778.0]

Interior-point solution summary
  Problem status  : PRIMAL_AND_DUAL_FEASIBLE
  Solution status : OPTIMAL
  Primal.  obj: 0.0000000000e+00    nrm: 2e+07    Viol.  con: 3e-05    var: 0e+00    barvar: 0e+00  
  Dual.    obj: 2.9627699405e-08    nrm: 6e+04    Viol.  con: 2e-18    var: 6e-11    barvar: 5e-12  
None
[24, 1, 10]=True
